In [1]:
import pandas as pd
import requests
import json
import glob
import itertools
import re

In [2]:
from sqlalchemy import create_engine

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning) 

In [5]:
url = 'https://www.dooinauction.com/auction/search/?km_status=ing&current_page=1&records_per_page=100'
driver = webdriver.Chrome(executable_path='chromedriver.exe')
driver.get(url)
driver.implicitly_wait(30)
soup = BeautifulSoup(driver.page_source, 'lxml')
driver.close()
last = int(soup.find_all('a','page-link')[-2].text)
print(last)

115


In [6]:
data = pd.DataFrame()

for p in range(last):
    p+=1
    driver = webdriver.Chrome(executable_path='chromedriver.exe')
    url = f'https://www.dooinauction.com/auction/search/?km_status=ing&current_page={p}&records_per_page=100'
    # url = f'https://www.dooinauction.com/auction/search/?km_status=ing&addr1%5B%5D=%EC%84%9C%EC%9A%B8&addr1%5B%5D=%EA%B2%BD%EA%B8%B0&current_page={p}&records_per_page=100'
    driver.get(url)
    print(url)
    driver.implicitly_wait(30)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    items = soup.find_all("div","list-product")
    # if len(items) ==0:
    #     print("BREAK")
    #     break
        
    # driver.close()
    df = pd.DataFrame()
    for i in items:
        d = pd.DataFrame()
        one = i.find_all('span',{'address-txt'})
        d['경매번호'] = [one[0].text.replace('지도 보기 ','')]
        d['주소지'] = [one[1].text]
        try:
            d['x'] = [(str(one[0]).split('urlX=')[1].split('&')[0])]
            d['y'] = [(str(one[0]).split('urlY=')[1].split('"')[0])]
        except IndexError:
            d['x'] = [""]
            d['y'] = [""]
        except ValueError:
            d['x'] = [""]
            d['y'] = [""]
        two = i.find_all('div',{'court-txt'})
        d['종류']  = [two[0].text]
        try:
            three = i.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['size-txt'])
            d['면적'] = [three[0].text]
        except IndexError:
            d['면적'] = ['']
        
        four = i.find_all('div','price-value')
        try:
            d['감정가'] = [int(four[0].text.replace(',',""))]
            d['입찰가'] = [int(four[1].text.replace(',',""))]
        except IndexError:
            d['감정가'] = [0]
            d['입찰가'] = [0]
        
        five = i.find_all('div', 'date-day')        
        d['입찰일'] = [five[0].text]
        
        six = i.find_all('input',value=True)[0]['value']
        try:
            d['링크'] = [six]
        except IndexError:
            d['링크'] = [0]
        df = df.append(d)
        
    data = data.append(df)
    

https://www.dooinauction.com/auction/search/?km_status=ing&current_page=1&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=2&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=3&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=4&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=5&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=6&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=7&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=8&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=9&records_per_page=100
https://www.dooinauction.com/auction/search/?km_status=ing&current_page=10&records_per_page=100
https://www.dooinauction.com/auction/search/?km_s

In [13]:
# data['감정가'] = data['감정가'].str.replace(',','')

In [19]:
data['크기'] = 10

data['x'] = pd.to_numeric(data['x'],errors='coerce')
data['y'] = pd.to_numeric(data['y'],errors='coerce')

# data['감정가']=data['감정가'].apply(lambda x : "{:,}".format(x))
# data['입찰가']=data['입찰가'].apply(lambda x : "{:,}".format(x))

data['감정가'] = pd.to_numeric(data['감정가'],errors='coerce')
data['입찰가'] = pd.to_numeric(data['입찰가'],errors='coerce')

data['감정가']=data['감정가'].apply("{:,}".format)
data['입찰가']=data['입찰가'].apply("{:,}".format)

data['링크'] = data["링크"].apply(lambda x: '<a href="https://www.dooinauction.com/auction/search/?auction_id={0}">link</a>'.format(x))

In [32]:
data.to_excel('data.xls',index=False)

data.to_csv("data.csv",encoding ='euc_kr',index=False)